# Time Scheduler for Campus Classes

Time scheduler is a `Generative-AI` based app, developed in `langchain` using OpenAI `gpt-4` model. It takes subjects and students data in CSV format and generates time schedule for each individual student.

In [27]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [28]:
import csv

def read_data_from_csv(filename):
    data = []

    with open(filename, 'r') as csvfile:
        return list(csv.DictReader(csvfile))

In [29]:
subjects_data = read_data_from_csv("subjects.csv")
students_data = read_data_from_csv("students.csv")

In [30]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(model="gpt-4", temperature=0)

In [31]:
system_message_prompt = SystemMessagePromptTemplate.from_template(
    "You are an expert university class scheduler with proficiency in interpreting JSON."
)
human_message_prompt = HumanMessagePromptTemplate.from_template(
"""
The following subjects` data contains subject names, the total number of classes to be attended by a student, and the available class slots for each subject.
{subjects_data}

Here is the data for a specific student, including his registered subjects and considerations. Your task is to schedule classes for these subjects.
{student}

The number of classes per week for each subject must be the same as mentioned in the data. You must make sure that classes should not overlap. If there is a conflict between the student's considerations and the scheduling, ignore the consideration. Include a brief comment at the end regarding the extent to which the considerations have been met, specifying the subject names.

The timetable should be prepared in the following format:
StudentName:
Subject1:
Classes list
Subject2:
Classes list
Subject3:
Classes list
...
Remember that classes must not overlap, ignore consideration when needed.
"""
)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [32]:
for student in students_data:
    response = chain.run(subjects_data=subjects_data, student=student)
    print(response, '\n')

Here is the class schedule for John Doe:

StudentName: John Doe

Subject1: Mathematics
Classes list:
- Tuesday 1:00 PM - 2:30 PM
- Tuesday 3:00 PM - 4:30 PM
- Thursday 1:00 PM - 2:30 PM

Subject2: Science
Classes list:
- Tuesday 9:00 AM - 10:30 AM
- Thursday 11:00 AM - 12:30 PM
- Tuesday 1:00 PM - 2:30 PM

Subject3: History
Classes list:
- Wednesday 1:00 PM - 2:30 PM
- Wednesday 3:00 PM - 4:30 PM

The student's consideration to avoid classes on Mondays and Fridays has been met for all subjects. However, there is a conflict in the schedule between Mathematics and Science on Tuesday 1:00 PM - 2:30 PM. In this case, the consideration was ignored to ensure that the number of classes per week for each subject is met. 

Here is the class schedule for Jane Smith:

StudentName: Jane Smith

Subject1: English
Classes list:
- Tuesday 1:00 PM - 2:30 PM
- Wednesday 9:00 AM - 10:30 AM
- Thursday 1:00 PM - 2:30 PM

Subject2: Mathematics
Classes list:
- Monday 9:00 AM - 10:30 AM
- Tuesday 3:00 PM - 4: